# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [88]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
#cities.set_index('Metropolitan area', inplace=True)
cities.rename(columns = {'Population (2016 est.)[8]':'Population'}, inplace=True)
cities[['Metropolitan area','NFL','MLB','NBA','NHL']] = cities[['Metropolitan area','NFL','MLB','NBA','NHL']].replace(['\[.*\]+','(?<=\w)([A-Z])',',.*','\–.*','\—'],['',r' \1','','',''], regex=True)
nhl_df = nhl_df.loc[(nhl_df['year']==2018)]
nhl_df = nhl_df[~nhl_df.GP.str.contains("Division")]
nhl_df = nhl_df[['team', 'W','L']]
nhl_df['team']=nhl_df['team'].replace('\*', '', regex=True)
cols = ['Metropolitan area','NFL','MLB','NBA','NHL']
cities[cols] = cities[cols].apply(lambda x: x.str.strip())
cols = ['team', 'W', 'L']
nhl_df[cols] = nhl_df[cols].apply(lambda x: x.str.strip())
nhl_df['Team'] = nhl_df['team'].apply(lambda x: x.split(' ')[-1])
nhl_df['Team'] = nhl_df['Team'].replace(['Leafs', 'Wings', 'Jackets', 'Knights'],['Maple Leafs','Red Wings','Blue Jackets', 'Golden Knights'])
df = cities[['Metropolitan area', 'NHL','Population']]
df = df.replace('',np.nan).dropna()
df.rename(columns = {"NHL":"Team"}, inplace=True)
df = df.set_index(['Metropolitan area','Population']).apply(lambda x: x.str.split(' ').explode()).reset_index()
df['Team'] = df['Team'].replace(['Leafs', 'Wings', 'Jackets', 'Knights'],['Maple Leafs','Red Wings','Blue Jackets', 'Golden Knights'])

nhl = pd.merge(nhl_df, df, how='inner', on='Team')
nhl['ratio'] = pd.to_numeric(nhl['W'])/(pd.to_numeric(nhl['L'])+pd.to_numeric(nhl['W']))
nhl.drop(['W', 'L'], axis=1, inplace=True)
data = nhl.groupby('Metropolitan area').agg({'Population':np.max, 'ratio':np.mean})

def nhl_correlation(): 
    
    
    population_by_region = pd.to_numeric(data['Population']) # pass in metropolitan area population from cities
    win_loss_by_region = data['ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    corr,pval = stats.pearsonr(population_by_region, win_loss_by_region)
    return corr
nhl_correlation()

0.012486162921209907

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [90]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
#cities=pd.read_html("assets/wikipedia_data.html")[1]
#cities=cities.iloc[:-1,[0,3,5,6,7,8]]
nba_df = nba_df.loc[(nba_df['year']==2018)]
nba_df = nba_df[['team', 'W/L%']]
nba_df['team']=nba_df['team'].replace(['\*.*','\(.*\)'], ['',''], regex=True)
cols = ['team', 'W/L%']
nba_df[cols] = nba_df[cols].apply(lambda x: x.str.strip())
nba_df['Team'] = nba_df['team'].apply(lambda x: x.split(' ')[-1])
nba_df['Team'] = nba_df['Team'].replace(['Blazers'],['Trail Blazers'])
df = cities[['Metropolitan area', 'NBA','Population']]
df = df.replace('',np.nan).dropna()
df.rename(columns = {"NBA":"Team"}, inplace=True)
df = df.set_index(['Metropolitan area','Population']).apply(lambda x: x.str.split(' ').explode()).reset_index()
df['Team'] = df['Team'].replace(['Blazers'],['Trail Blazers'])
#print(nba_df.columns.values.tolist())
#print(df.head(50))
nba = pd.merge(nba_df, df, how='inner', on='Team')
#print(temp.columns.values.tolist())
nba[['Population','W/L%']] = nba[['Population','W/L%']].apply(pd.to_numeric, errors='coerce')
data = nba.groupby('Metropolitan area').agg({'Population':np.max, 'W/L%':np.mean})

def nba_correlation():
    
    
    population_by_region = data['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = data['W/L%'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)
    return corr
nba_correlation()

-0.17636350642182935

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [91]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
#cities=pd.read_html("assets/wikipedia_data.html")[1]
#cities=cities.iloc[:-1,[0,3,5,6,7,8]]
mlb_df = mlb_df.loc[(mlb_df['year']==2018)]
mlb_df = mlb_df[['team', 'W-L%']]
#nba_df['team']=nba_df['team'].replace(['\*.*','\(.*\)'], ['',''], regex=True)
cols = ['team']
mlb_df[cols] = mlb_df[cols].apply(lambda x: x.str.strip())
mlb_df['Team'] = mlb_df['team'].apply(lambda x: x.split(' ')[-1])
mlb_df['Team'] = mlb_df['Team'].replace(['Jays'],['Blue Jays'])
mlb_df['Team'].iloc[0] = 'Red Sox'
mlb_df['Team'].iloc[8] = 'White Sox'
df = cities[['Metropolitan area', 'MLB','Population']]
df = df.replace('',np.nan).dropna()
df.rename(columns = {"MLB":"Team"}, inplace=True)
df = df.set_index(['Metropolitan area','Population']).apply(lambda x: x.str.split(' ').explode()).reset_index()
df['Team'] = df['Team'].replace(['Jays','Red','White'],['Blue Jays','Red Sox','White Sox'])

mlb = pd.merge(mlb_df, df, how='inner', on='Team')
#print(temp.columns.values.tolist())
mlb[['Population','W-L%']] = mlb[['Population','W-L%']].apply(pd.to_numeric, errors='coerce')
data = mlb.groupby('Metropolitan area').agg({'Population':np.max, 'W-L%':np.mean})

def mlb_correlation(): 
    
    
    
    population_by_region = data['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = data['W-L%'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)
    return corr

mlb_correlation()

0.15003737475409495

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [92]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
#cities=pd.read_html("assets/wikipedia_data.html")[1]
#cities=cities.iloc[:-1,[0,3,5,6,7,8]]
nfl_df = nfl_df.loc[(nfl_df['year']==2018)]
nfl_df = nfl_df[~nfl_df.L.str.contains("FC")]
nfl_df = nfl_df[['team', 'W-L%']]
nfl_df['team']=nfl_df['team'].replace(['\*', '\+'], ['',''], regex=True)
cols = ['team', 'W-L%']
nfl_df[cols] = nfl_df[cols].apply(lambda x: x.str.strip())
nfl_df['Team'] = nfl_df['team'].apply(lambda x: x.split(' ')[-1])
#nhl_df['Team'] = nhl_df['Team'].replace(['Leafs', 'Wings', 'Jackets', 'Knights'],['Maple Leafs','Red Wings','Blue Jackets', 'Golden Knights'])
df = cities[['Metropolitan area', 'NFL','Population']]
df = df.replace('',np.nan).dropna()
df.rename(columns = {"NFL":"Team"}, inplace=True)
df = df.set_index(['Metropolitan area','Population']).apply(lambda x: x.str.split(' ').explode()).reset_index()
#df['Team'] = df['Team'].replace(['Leafs', 'Wings', 'Jackets', 'Knights'],['Maple Leafs','Red Wings','Blue Jackets', 'Golden Knights'])
#print(df.head(50))

nfl = pd.merge(nfl_df, df, how='inner', on='Team')
#print(temp.columns.values.tolist())
nfl[['Population','W-L%']] = nfl[['Population','W-L%']].apply(pd.to_numeric, errors='coerce')
data = nfl.groupby('Metropolitan area').agg({'Population':np.max, 'W-L%':np.mean})

def nfl_correlation(): 
    
    
    population_by_region = data['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = data['W-L%'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)
    return corr
nfl_correlation()

0.004282141436392957

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [109]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

#mlb_df=pd.read_csv("assets/mlb.csv")
#nhl_df=pd.read_csv("assets/nhl.csv")
#nba_df=pd.read_csv("assets/nba.csv")
#nfl_df=pd.read_csv("assets/nfl.csv")
#cities=pd.read_html("assets/wikipedia_data.html")[1]
#cities=cities.iloc[:-1,[0,3,5,6,7,8]]
#nhl_df['ratio'] = pd.to_numeric(nhl_df['W'])/(pd.to_numeric(nhl_df['L'])+pd.to_numeric(nhl_df['W']))
#nhl_df.drop(['W', 'L'], axis=1, inplace=True)
nhl = nhl.groupby('Metropolitan area').agg({'ratio': np.mean})
nba = nba.groupby('Metropolitan area').agg({'W/L%': np.mean})
mlb = mlb.groupby('Metropolitan area').agg({'W-L%': np.mean})
nfl = nfl.groupby('Metropolitan area').agg({'W-L%': np.mean})
nba_nhl = pd.merge(nba,nhl,how='inner',on='Metropolitan area')
nba_nfl = pd.merge(nba,nfl,how='inner',on='Metropolitan area')
nba_mlb = pd.merge(nba,mlb,how='inner',on='Metropolitan area')
nhl_nfl = pd.merge(nhl,nfl,how='inner',on='Metropolitan area')
nhl_mlb = pd.merge(nhl,mlb,how='inner',on='Metropolitan area')
mlb_nfl = pd.merge(mlb,nfl,how='inner',on='Metropolitan area')
#print(mlb_nfl.head())

def sports_team_performance():
    nba_nhl_test, pval1 = stats.ttest_rel(nba_nhl['W/L%'], nba_nhl['ratio'])
    nba_nfl_test, pval2 = stats.ttest_rel(nba_nfl['W/L%'], nba_nfl['W-L%'])
    nba_mlb_test, pval3 = stats.ttest_rel(nba_mlb['W/L%'], nba_mlb['W-L%'])
    nhl_nfl_test, pval4 = stats.ttest_rel(nhl_nfl['ratio'], nhl_nfl['W-L%'])
    nhl_mlb_test, pval5 = stats.ttest_rel(nhl_mlb['ratio'], nhl_mlb['W-L%'])
    mlb_nfl_test, pval6 = stats.ttest_rel(mlb_nfl['W-L%_x'], mlb_nfl['W-L%_y'])
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    p_values.iloc[0] = [np.nan, pval2, pval4, pval6]
    p_values.iloc[1] = [pval2, np.nan, pval1, pval3]
    p_values.iloc[2] = [pval4, pval1, np.nan, pval5]
    p_values.iloc[3] = [pval6, pval3, pval5, np.nan]
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.937509,0.030318,0.803459
NBA,0.937509,NaN,0.022386,0.949566
NHL,0.030318,0.022386,NaN,0.000703
MLB,0.803459,0.949566,0.000703,NaN
